In [25]:
import pandas as pd
import numpy as np
import pickle as p
import time

In [26]:
data_all = pd.read_csv('../data_all.csv')

# Test with Subset of Data
np.random.seed(43)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=10, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

test_project_id = 'test_6'

### `/train`

In [27]:
import requests
import json
from threading import Thread

url = 'https://fillet.azurewebsites.net/train/'
# url = 'http://localhost:5000/train/'

data_subset.to_parquet('data_subset.parquet')
files = {'data': open('data_subset.parquet', 'rb')}

payload = {'cv_acc': True, 'project_id': test_project_id, 'modeltype': 'default'}

Thread(target=requests.post,
       args=(url, ),
       kwargs={
           'data': payload,
           'files': files
       }).start()

time.sleep(10)
files['data'].close()

### `/query_progress`

In [ ]:
import requests
import json

url = 'https://fillet.azurewebsites.net/query_progress/'
# url = 'http://localhost:5000/query_progress/'


payload = {'project_id': test_project_id}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=3)

print('% Complete:',round(r.json()['pct_complete'],3))
print('CV % Complete:',round(r.json()['cv_progress'],3))
print('CV Done:',r.json()['cv_done'])

### `/batch_query_progress`

In [6]:
import requests
import json

url = 'https://fillet.azurewebsites.net/batch_query_progress/'
# url = 'http://localhost:5000/batch_query_progress/'


payload = {'project_id_ls': ['8b59136f-5ca6-4a10-9e65-bb4171e887bd','c759e900-fbc7-4b59-82db-1fd8a81a0f2c']}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=3)

print(r.json())

{'8b59136f-5ca6-4a10-9e65-bb4171e887bd': {'cv_done': 1, 'cv_progress': 100.0, 'fi_done': 1, 'pct_complete': 100.0, 'project_items': [1, 2, 11, 12, 2571, 3601, 21, 22, 2581, 2588, 2589, 5662, 31, 5666, 5690, 1615, 6750, 101, 102, 4725, 4728, 1152, 1153, 16000, 16001, 3716, 3717, 3718, 16002, 16003, 3721, 3722, 3723, 5771, 2701, 3726, 3727, 3728, 16016, 16017, 3731, 3732, 3733, 16018, 16019, 3736, 3737, 3738, 3742, 3743, 2721, 4265, 1195, 2731, 4267, 4268, 4270, 2736, 4272, 4273, 1203, 1204, 1205, 3766, 3767, 3768, 7159, 3771, 3772, 3773, 5820, 196, 197, 1221, 200, 201, 202, 203, 204, 1225, 1230, 1231, 1232, 2256, 2257, 2258, 2259, 3816, 3817, 3818, 3821, 3822, 3823, 2313, 7434, 2315, 2316, 281, 284, 7965, 7966, 7967, 2867, 4409, 4410, 4411, 2887, 5469, 350, 7008, 7009, 7010, 4975, 4976, 4977, 4978, 5501, 7041, 7575, 7078, 7079, 7080, 5060, 2501, 7108, 7109, 7110, 7113, 7114, 7115, 5068, 5069, 5581, 2511, 5073, 5074, 7128, 2521, 7129, 1502, 1503, 1504, 2531, 3045, 3046, 3047, 1002, 1003,

### `/get_cv_results`

In [22]:
import requests
import json

url = 'https://fillet.azurewebsites.net/get_cv_results/'
# url = 'http://localhost:5000/get_cv_results/'


payload = {'project_id': test_project_id}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=5)
pd.DataFrame().from_dict((json.loads(r.json())))

,item_id,avg_sales,r2_score,mae_score,mpe_score,rmse_score
0,3743.0,132.480972,0.817921,23.323702,0.176053,31.143002
1,1230.0,8.996674,0.438134,4.181039,0.464732,5.732625
2,3766.0,91.448227,0.778293,15.238964,0.166640,20.912381
3,2531.0,318.983582,0.496373,82.516815,0.258687,109.153951
4,202.0,2110.440186,0.783663,332.939138,0.157758,462.025116
5,2589.0,161.616043,0.703087,35.876050,0.221983,47.594348
6,3726.0,74.383606,0.663344,16.482786,0.221592,21.208354
7,3732.0,126.142509,0.669734,22.727081,0.180170,29.528547
8,1195.0,129.492813,0.601521,27.792955,0.214629,34.932227
9,16019.0,109.613182,0.663948,26.357253,0.240457,34.293778


In [127]:
r.json()

{'status': 'incomplete'}

### `/optimize`

In [10]:
import json
with open('constraints_2.json') as f:
    constraints = json.load(f)
    
import requests
import json
from threading import Thread

url = 'https://fillet.azurewebsites.net/optimize/'
# url = 'http://localhost:5000/optimize/'
    
payload = {'project_id': test_project_id,
           'constraints': constraints, # two parts, first part constraint, second part price range limits
           'population': 300,
           'max_epoch': 20
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
# r = requests.post(url, data=payload, headers=headers, timeout=240)

Thread(target=requests.post,
       args=(url, ),
       kwargs={
           'data': payload,
           'headers': headers
       }).start()

### /get_opti_results

In [12]:

import requests
import json

url = 'https://fillet.azurewebsites.net/get_opti_results/'
# url = 'http://localhost:5000/get_opti_results/'


payload = {'project_id': test_project_id}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=5)
r.json()






{'price_cols': ['Price_202',
  'Price_1195',
  'Price_1230',
  'Price_2531',
  'Price_2589',
  'Price_3726',
  'Price_3732',
  'Price_3743',
  'Price_3766',
  'Price_16019'],
 'report': [18067.070645713808, 0.0, 0.0],
 'report_info': ['estimated revenue of the optimized price',
  'number of hard constraints (including price ranges) violated',
  'number of soft constraints (preferences) violated'],
 'result': [2.15,
  3.6,
  3.6500000000000004,
  6.6000000000000005,
  11.100000000000001,
  10.850000000000001,
  12.55,
  11.950000000000001,
  10.3,
  14.15]}

### `/predict/`

In [38]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

# print('Input Prices Quantities:')
# a_input

In [40]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'https://fillet.azurewebsites.net/predict/'
# url = 'http://localhost:5000/predict/'

payload = {'prices': a_input, 'project_id': test_project_id, 'modeltype': 'default'}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

,Estimated,Actual
Qty_102,219.0,225.0
Qty_1195,99.0,98.0
Qty_1221,240.0,217.0
Qty_1230,6.0,6.0
Qty_16017,20.0,17.0
Qty_16018,111.0,143.0
Qty_16019,137.0,164.0
Qty_201,40.0,43.0
Qty_202,2801.0,2949.0
Qty_204,17.0,17.0


### `/get_feature_importances`

In [15]:
import requests
import json
import pandas as pd

url = 'https://fillet.azurewebsites.net/get_feature_importances/'
# url = 'http://localhost:5000/get_feature_importances/'

payload = {'project_id': test_project_id}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=240)
pd.DataFrame(r.json()['202']).sort_values('importance', ascending=False)

,feature_name,importance
6,Price_3726,0.157354
3,Price_202,0.123959
4,Price_2531,0.117571
5,Price_2589,0.117251
1,Price_1230,0.104129
7,Price_3732,0.093999
9,Price_3766,0.093018
8,Price_3743,0.072451
0,Price_1195,0.060886
2,Price_16019,0.059381


In [24]:
import requests
import json
import pandas as pd

url = 'https://fillet.azurewebsites.net/get_elasticity_estimates/'
# url = 'http://localhost:5000/get_elasticity_estimates/'

payload = {'project_id': test_project_id}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=240)
pd.DataFrame(r.json()['16019'])

,baseline_qty,decrease_pct_chng,decrease_qty,increase_pct_chng,increase_qty
Qty_1195,125.0,-0.032000,121.0,-0.032000,121.0
Qty_1230,9.0,-0.111111,8.0,-0.111111,8.0
Qty_16019,149.0,-0.322148,101.0,0.033557,154.0
Qty_202,2097.0,0.030043,2160.0,0.013829,2126.0
Qty_2531,256.0,0.042969,267.0,0.105469,283.0
Qty_2589,128.0,0.046875,134.0,0.109375,142.0
Qty_3726,61.0,0.147541,70.0,-0.016393,60.0
Qty_3732,121.0,0.132231,137.0,0.115702,135.0
Qty_3743,122.0,0.106557,135.0,0.180328,144.0
Qty_3766,77.0,0.246753,96.0,0.246753,96.0


### /detect_conflict


In [3]:
import requests
import json

url = 'https://fillet.azurewebsites.net/detect_conflict/'
# url = 'http://localhost:5000/detect_conflict/'
with open('constraints_2.json') as f:
    constraints = json.load(f)

payload = {'constraints': constraints
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [4]:
r.json()

{'conflict': 'No conflict'}